[Node 68: Webprogrammierung](http://www-static.etp.physik.uni-muenchen.de/kurs/Computing/python2/node68.html)

Navigation:

**Next:** [Web-Framework Django](node69.ipynb) **Up:** [Web-Programmierung und Django](node67.ipynb) **Previous:** [Web-Programmierung und Django](node67.ipynb)

##  Web-Programmierung
Aus Performanzgründen werden Webserver normalerweise in C/C++ geschrieben, wie z.B. Apache oder [lighttpd](https://www.lighttpd.net/). Nicht immer ist eine solche Performanz nötig wie z.B. in eingebetteten Systemen. 

Die Python-Standardbibliothek bietet zwei von   <font color=#0000e6> ``Socket.Server.TCPServer``</font>  abgeleitete Klassen für einen Webserver:   <font color=#0000e6> ``SimpleHTTPServer``</font>  und    <font color=#0000e6> ``CGIHTTPServer``</font> . Beide Klassen basieren auf dem   <font color=#0000e6> ``BaseHTTPServer``</font> .   

Der einfachste Webserver:

In [ ]:
import BaseHTTPServer
server_address = ("", 9090)
handler_class  = BaseHTTPServer.BaseHTTPRequestHandler
server_class   = BaseHTTPServer.HTTPServer
server = server_class(server_address, handler_class)
server.serve_forever()

 Führen Sie dieses Programm aus und rufen Sie in einem Webbrower folgende Addresse auf: [http://localhost:9090/](http://localhost:9090/). 

Das funktioniert noch nicht richtig: Der Server kann die   <font color=#0000e6> ``GET``</font>-Methode nicht interpretieren, da die   <font color=#0000e6> ``handle``</font>-Methode des    <font color=#0000e6> ``BaseHTTPRequestHandler``</font>  keine   <font color=#0000e6> ``do_GET``</font>-Methode finden konnte.    

Eine entsprechende Erweiterung ist nötig:

In [ ]:
import BaseHTTPServer
class MyHandler(BaseHTTPServer.BaseHTTPRequestHandler):
    def do_GET(self):
        print "Got GET Request from", self.client_address
        self.wfile.write('Sorry, I do not speak HTTP. Go away.\r\n')
server_address = ("", 9091)
handler_class  = MyHandler
server_class   = BaseHTTPServer.HTTPServer
server = server_class(server_address, handler_class)
server.serve_forever()

Jetzt funktioniert der Aufruf von http://localhost:9091/.

(Hinweis: Wir verwenden hier wie schon zuvor jedes Mal einen anderen Port, um [dieses Problem](https://dev.to/dechamp/the-dreaded-bind-address-already-in-use-kill-it-583l) zu umgehen. Es gibt ja genügend davon. Eine andere, elegantere Lösung ist es, die Ausnahme aufzufangen und den Socket wieder freizugeben -- siehe unten.)

Dieses Beispiel kann man nun z.B. zu einem einfachen Taschenrechner erweitern. Dieser akzeptiert Anfragen der Form "<tt>/{add,sub,mul,div}/num1/num2</tt>", die in der URL übergeben werden. Diese Anfrage erlaubt, zwei Zahlen <tt>num1</tt> und <tt>num2</tt> zu addieren, subtrahieren etc. 

Das Beispiel führt auch eine Reihe von Überprüfungen durch, um sicherzustellen, dass die Anfrage gültig ist. Dies ist unerlässlich bei Services, die weltoffen sind, um keine Angriffsfläche für Missbrauch zu bieten. (Typischerweise wird ein Server, den Ihr auf Eurem eigenen Rechner startet, nicht ohne weiteres weltweit, sondern nur von Eurem Heimnetz aus zugreifbar sein.)

Zusätzlich ist hier ``server.serve_forever()`` in einen ``try-except``-Block verpackt, um den Server sauber stoppen (und den Port wieder freigeben) zu können.

In [ ]:
import http.server

class CalcHandler(http.server.BaseHTTPRequestHandler):

  def do_GET(self):
    path = self.path

    lst = path.split("/")
    if len(lst) != 4:
      self.send_response(403)
      self.end_headers()
      self.wfile.write(b"Illegal syntax. Use /{add,sub,mul,div}/num1/num2\r\n")
      return

    dummy, op, arg1, arg2 = lst

    if op not in ("add", "sub", "mul", "div"):
      self.send_response(403)
      self.end_headers()
      self.wfile.write(b"Illegal operation: %s\r\n" % op)
      return

    try:
      numarg1 = float(arg1)
      numarg2 = float(arg2)
    except ValueError:
      self.send_response(403)
      self.end_headers()
      self.wfile.write(b"Numerical arguments expected\r\n")
      return

    if op == "add":
      result = numarg1 + numarg2
    elif op == "sub":
      result = numarg1 - numarg2
    elif op == "mul":
      result = numarg1 * numarg2
    elif op == "div":
      if numarg2 == 0:
        result = "NaN"
      else:
        result = numarg1 / numarg2

    self.send_response(200)
    self.end_headers()
    self.wfile.write(str(result).encode())


def run_server(port=9092):
  server_class = http.server.HTTPServer
  server_address = ("", port)
  handler_class = CalcHandler

  server = server_class(server_address, handler_class)
  try:
    print("Starting server...")
    server.serve_forever()
  except KeyboardInterrupt:
    print("Stopping server...")
    server.socket.close()
  except: # something else went wrong
    raise

if __name__ == "__main__":
  run_server()

Dieser ist erreichbar auf http://localhost:9092.

---

Etwas mehr Webserver–Funktionalität bietet der   <font color=#0000ff> **SimpleHTTPServer**</font> , damit kann man Verzeichnisse anzeigen und  Dateien übertragen:

In [ ]:
import http.server
            
def run_server(port=9093):
    server_class   = http.server.HTTPServer
    handler_class  = http.server.SimpleHTTPRequestHandler
    server_address = ("", port)
    
    server = server_class(server_address, handler_class)
    try:
        server.serve_forever()
    except KeyboardInterrupt:
        server.socket.close()
    except:
        raise
    
if __name__ == '__main__':
    run_server()

Aufrufadresse: http://localhost:9093

Der nächste Schritt ist, nicht nur Inhalte anzuzeigen, sondern auch Skripte auszuführen.
Am einfachsten geht das mit sogenannten <font color=#0000ff> **CGI-Skripten**</font>  unter Verwendung von `CGIHTTPServer`. Mit CGI-Skripten, die im  <font color=#0000e6> ``cgi-bin``</font>-Unterverzeichis des Webservers plaziert werden, können Programme auf dem Webserver ausgeführt und die Ergebisse an den Klienten übertragen werden. Allerdings ist das unter Sicherheitsaspekten heikel. Die Skripte können in beliebigen Programmiersprachen geschrieben sein, nicht nur Python (solange es serverseitig unterstützt ist). 

Ein   <font color=#0000ff> **CGIHTTPServer**</font>:

In [ ]:
import http.server
            
def run_server(port=9094):
    server_class   = http.server.HTTPServer
    handler_class  = http.server.CGIHTTPRequestHandler
    handler_class.cgi_directories = ['/source', '/source/cgi-bin']
    server_address = ("", port)
    
    server = server_class(server_address, handler_class)
    try:
        server.serve_forever()
    except KeyboardInterrupt:
        server.socket.close()
    except:
        raise
        
if __name__ == '__main__':
    run_server()

Folgendes CGI-Skript plazieren wir mit Namen <tt>cgiprintenv.py</tt> in das Unterverzeichnis <tt>source/cgi-bin/</tt>:
```python
#!/usr/bin/env python3

import os
from sys import stdout

stdout.write("Content-type: text/plain\r\n\r\n")

for key in sorted(os.environ.keys()):
    print("%s=%s" % (key, os.environ[key]))
```
und machen es ausführbar (`chmod +x cgiprintenv.py`).

Dann kann man es einem Webbrower unter folgender Addresse aufrufen: http://localhost:9094/source/cgi-bin/cgiprintenv.py

 Ein weiterführende elegante CGI-Lösung ist der [**WSGI-Standard**](https://de.wikipedia.org/wiki/Web_Server_Gateway_Interface), auf den  hier aber nicht weiter eingegangen wird.  

